In [ ]:
### Import the package
import storypy as sp # check if this is used for something else
# import esmvaltool
# from esmvaltool.diag_scripts.shared import run_diagnostic, get_cfg, group_metadata
# from esmvaltool.diag_scripts.shared._base import _get_input_data_files
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
'''
To use the esmvaltool configuration file, we need to import the esmvaltool package.
This package is not available in the current environment.
The following code is an example of how to use the esmvaltool configuration file.
'''

'''
def parse_config(file):
    """Parse the settings file."""
    config = get_cfg(file)           
    config['input_data'] = _get_input_data_files(config)
    return config
'''

'''
# Load the configuration file
config= parse_config('/climca/people/ralawode/esmvaltool_output/full_storyline_analysis_complete_20240923_140137/run/multiple_regression_indices/multiple_regresion/settings.yml')'

cs.main_esmval(config, user_config)
'''

This configuration dictionary is used to set up the processing parameters for the climate data analysis tool. Each key in the dictionary configures a specific aspect of the data processing pipeline, from specifying file directories to setting analysis periods and regions.

## Keys and Descriptions
- data_dir
    - Type: String
    - Description:
        - The directory path where the source CMIP6 netCDF files are stored.
    - Example: '/climca/data/cmip6-ng'

- work_dir
    - Type: String
    - Description:
        - The directory path where intermediate processing outputs and combined netCDF files are written.
    - Example: '/climca/people/storylinetool/test_user/work_dir'

- plot_dir
    - Type: String
    - Description:
        - The directory path where the generated plots (e.g., time series plots) will be saved.
    - Example: '/climca/people/storylinetool/test_user/plot_dir'

- var_name
    - Type: List of strings
    - Description:
        - A list of variable names to be processed. These variable names correspond to the variables available in the dataset (e.g., precipitation (pr), sea level pressure (psl)).
    - Example: ['pr', 'psl']

- exp_name
    - Type: String
    - Description:
        - The experiment name (scenario) to filter scenario files (e.g., climate change scenario identifier).
    - Example: 'ssp585'

- freq
    - Type: String
    - Description:
        - The frequency of the data, such as monthly ('mon').
    - Example: 'mon'

- grid
    - Type: String
    - Description:
        - The grid resolution of the dataset (e.g., 'g025' for a 0.25° grid).
    - Example: 'g025'

- region_method
    - Type: String
    - Description:
        - The method to define the region for spatial analysis. For example, using a bounding box method ('box').
    - Example: 'box'

- period1
    - Type: List of strings
    - Description:
        - The first time period used for the baseline climatology (e.g., historical period). The list contains the start and end years as strings.
    - Example: ['1950', '1979']

- period2
    - Type: List of strings
    - Description:
        - The second time period used for future projections. This list also contains the start and end years as strings.
    - Example: ['2070', '2099']

- region_id
    - Type: Integer
    - Description:
        - An identifier for the region being analyzed. This can be used to apply region-specific processing or look up additional metadata.
    - Example: 18

- season
    - Type: List of integers (or tuple)
    - Description:
        - The months representing the season for which the analysis is performed. This can be provided as a list or a tuple.
    - Example: [11, 12, 1, 2, 3]
    - Note: You can also supply a tuple, e.g., (12, 1, 2).

- region_extents
    - Type: List of tuples
    - Description:
        - A list of tuples, where each tuple defines the geographical bounding box for a region. The tuple values represent (latitude_min, latitude_max, longitude_min, longitude_max).
    - Example: [(30, 45, -10, 40), (45, 55, 5, 20)]

In [ ]:
from storypy.preprocess import DirectProcessor

user_config = dict(
        data_dir='/climca/people/ralawode/cmip_test',
        work_dir='/climca/people/storylinetool/test_user/work_dir',
        plot_dir='/climca/people/storylinetool/test_user/plot_dir',
        var_name=['psl'],
        exp_name='ssp585',
        freq='mon',
        grid='g025',
        region_method='box',
        period1 = ['1950', '1979'],
        period2 = ['2070', '2099'],
        region_id=18,
        season=(11, 12, 1, 2, 3, 4),  # Now provided as a tuple of months
        region_extents=[(30, 45, -10, 40), (45, 55, 5, 20)],# (-40, 0, -60, -30)],
        titles=["Region A", "Region B"]
    )

driver_config = dict(
        var_name=['psl'],            # <- actual variable names in NetCDF
        short_name=['test_ubi'],           # <- names for regression/CSV outputs
        period1=['1960', '1979'],
        period2=['2070', '2099'],
        season=[12,1,2],
        #box={'lat_min': -15, 'lat_max': 15, 'lon_min': -180, 'lon_max': 180}, # ta
        box={'lat_min': 50, 'lat_max': 70, 'lon_min': 40, 'lon_max': 70}, 
        # work_dir='/climca/people/storylinetool/test_user/driver_outputs'
        work_dir='/climca/people/storylinetool/test_user/driver_test_outputs'
    )

'''
Run the diagnostics:
--------------------
- Be sure to have created the work_dir and plot_dir
- Running the line below will save the ouput NetCDF file in the work_dir and the plots in the plot_dir.
'''
processor_target = DirectProcessor(user_config, driver_config)
# sp.main_direct(user_config)

In [ ]:
# This computes the target and saves the output in the work_dir
# If multiple variables are provided, it will compute the target for each variable and save a single file with all variables nested in the work_dir
processor_target.process_var()

In [ ]:
# This computes the drivers and saves the output in the work_dir
processor_target.process_driver()

Users could start from here and import the preprocessed example data that we stored in storypy (za_sh2017, mi2020, mo2023, or gh2023).

In [ ]:
# This process the driver outputs and return the results containing the regression coefficients
from storypy.compute import compute_drivers_from_netcdf

compute_drivers_from_netcdf(driver_config)

In [ ]:
# To compute the regression indices (i.e MLR), we can use the following function
from storypy.compute import run_regression
run_regression(user_config, driver_config)

In [ ]:
from regres import spatial_MLR
import os

def run_regression(preproc, user_config, regressor_csv_path):
    """
    Run spatial multiple linear regression (MLR) using a preprocessed NetCDF dataset and regressors CSV.
    
    Parameters:
        preproc (str): Path to the preprocessed NetCDF file.
        user_config (dict): Configuration dictionary containing keys like "work_dir".
        regressor_csv_path (str): Path to the CSV file containing regressors data.
    
    This function:
      1. Opens the preprocessed NetCDF file.
      2. Loads regressors from a CSV file.
      3. Finds common models between the dataset and the regressors.
      4. Subsets the dataset based on common models.
      5. Aligns the regressors DataFrame to the common models.
      6. Prepares the regressor names by inserting 'MEM' at the beginning.
      7. Instantiates spatial_MLR, sets up regression data, and performs the regression.
      8. Saves the regression output to the specified work directory.
    """

    target = user_config["target"]
   
    ds = xr.open_dataset(preproc)
    # Ensure the model coordinate is a string and stripped of any whitespace.
    ds_model_names = pd.Index(ds['model'].values.astype(str)).str.strip()
    
 
    regressors = pd.read_csv(regressor_csv_path, index_col=0)
    regressors.index = regressors.index.str.strip()  # Clean the index if necessary

    ds_unique = ds.groupby('model').first()
    common_models = list(regressors.index.intersection(ds_unique['model'].values))
    print("Common models:", common_models)
    
    # Subset and reindex using ds_unique.
    ds_subset = ds_unique.sel(model=common_models).reindex(model=common_models)
    
    target_var = user_config.get("target", "pr")
    target = ds_subset[target_var]

    regressors_aligned = regressors.loc[common_models]

    regressor_names = regressors_aligned.columns.insert(0, 'MEM')

    # Note: spatial_MLR should be defined/imported from your module.
    MLR = spatial_MLR()
    MLR.regression_data(target, regressors_aligned, regressor_names)

    output_path = os.path.join(user_config["work_dir"], 'regression_output')
    os.makedirs(output_path, exist_ok=True)
    os.chdir(output_path)
    MLR.perform_regression(output_path, 'pr')

Users could decide to start from here and import the preprocessed data that we stored in storypy.

In [ ]:
user_config = dict(
        data_dir='/climca/data/cmip6-ng',
        work_dir='/climca/people/storylinetool/test_user/work_dir',
        plot_dir='/climca/people/storylinetool/test_user/plot_dir',
        var_name=['pr'],
        target='pr'
    )

preproc_file = '/climca/people/storylinetool/test_user/work_dir/combined_changes.nc'
regressor_csv = '/climca/people/ralawode/esmvaltool_output/zappa_shepherd_CMIP6_20250115_193953/work/storyline_analysis/remote_drivers/remote_drivers/scaled_standardized_drivers.csv'

run_regression(preproc_file, user_config, regressor_csv)

In [ ]:
"""
import xarray as xr
import numpy as np
import pandas as pd
import os
import glob

def stand(dato):
    anom = (dato - np.mean(dato))/np.std(dato)
    return anom


def compute_drivers_from_netcdf(driver_config):
    work_dir = driver_config['work_dir']
    var_names = driver_config['var_name']
    short_names = driver_config.get('short_name', var_names)

    if len(var_names) != len(short_names):
        raise ValueError("Length of 'var_name' and 'short_name' must match.")

    var_map = dict(zip(short_names, var_names))
    driver_files = glob.glob(os.path.join(work_dir, "*.nc"))
    if not driver_files:
        raise FileNotFoundError(f"No .nc files found in {work_dir}")

    # Initialize structure
    model_set = set()
    regressor_values = {sn: {} for sn in short_names}

    for f in driver_files:
        ds = xr.open_dataset(f)

        if 'model' not in ds.coords:
            print(f"Skipping file without 'model' coord: {f}")
            continue

        for short_name in short_names:
            if short_name not in os.path.basename(f):
                continue  # Skip this file unless it matches the variable

            if short_name not in ds:
                print(f"Variable {short_name} not found in {f}, skipping.")
                continue

            for model in ds['model'].values:
                model = str(model)
                model_set.add(model)
                try:
                    val = ds[short_name].sel(model=model).mean().item()
                except Exception as e:
                    print(f"Could not extract {short_name} for model {model} in {f}: {e}")
                    val = np.nan
                regressor_values[short_name][model] = val

    # Build dataframe from collected values
    models = sorted(model_set)
    data = {sn: [regressor_values[sn].get(m, np.nan) for m in models] for sn in short_names}
    df_raw = pd.DataFrame(data, index=models)

    df_scaled = df_raw.copy()
    df_standardized = df_scaled.apply(stand, axis=0)

    out_dir = os.path.join(work_dir, "remote_drivers")
    os.makedirs(out_dir, exist_ok=True)
    df_raw.to_csv(os.path.join(out_dir, "drivers.csv"))
    df_scaled.to_csv(os.path.join(out_dir, "scaled_drivers.csv"))
    df_standardized.to_csv(os.path.join(out_dir, "scaled_standardized_drivers.csv"))

    print(f"Saved driver regressors to: {out_dir}")
"""

import xarray as xr
import numpy as np
import pandas as pd
import os
import glob

def stand(dato):
    """
    Standardize the data (z-score normalization)
    """
    anom = (dato - np.mean(dato)) / np.std(dato)
    return anom

def compute_drivers_from_netcdf(driver_config):
    """
    Compute driver values from preprocessed NetCDF files stored in driver_config['work_dir'].

    The function handles both variable extraction and global warming scaling.

    driver_config:
        var_name: list of variable names in NetCDF
        short_name: list of output variable names for regression/CSV
        work_dir: path where NetCDF files are stored
    """
    work_dir = driver_config['work_dir']
    var_names = driver_config['var_name']
    short_names = driver_config.get('short_name', var_names)

    if len(var_names) != len(short_names):
        raise ValueError("Length of 'var_name' and 'short_name' must match.")

    # Map short_name to var_name
    var_map = dict(zip(short_names, var_names))

    # Find all .nc files in the directory
    driver_files = glob.glob(os.path.join(work_dir, "*.nc"))
    if not driver_files:
        raise FileNotFoundError(f"No .nc files found in {work_dir}")

    # Load the global warming index file
    gw_file = os.path.join(work_dir, "remote_driver_gw.nc")
    if not os.path.exists(gw_file):
        raise FileNotFoundError("Missing required global warming file: remote_driver_gw.nc")
    gw_ds = xr.open_dataset(gw_file)

    if 'gw' not in gw_ds:
        raise ValueError("Global warming dataset must contain variable 'gw'")

    gw_vals = gw_ds['gw']
    if 'model' not in gw_vals.coords:
        raise ValueError("'gw' variable must have 'model' coordinate")

    models = gw_vals['model'].values.tolist()

    # Initialize structure for models and regressor values
    model_set = set()
    regressor_values = {sn: {} for sn in short_names}
    regressor_values_scaled = {sn: {} for sn in short_names}

    available_model_sets = {sn: set() for sn in short_names}

    # Iterate over each NetCDF file
    for f in driver_files:
        ds = xr.open_dataset(f)

        if 'model' not in ds.coords:
            print(f"Skipping file without 'model' coord: {f}")
            continue

        # For each short_name, check if it's in the file and process
        for short_name in short_names:
            if short_name not in os.path.basename(f):
                continue  # Skip if the file does not match the variable

            if short_name not in ds:
                print(f"Variable {short_name} not found in {f}, skipping.")
                continue

            # Extract data for each model in the dataset
            for model in models:
                model = str(model)
                model_set.add(model)

                try:
                    # Extract the mean value of the variable for the model
                    val = ds[short_name].sel(model=model).mean().item()
                    
                    # Get the global warming value for the model
                    gw_val = gw_vals.sel(model=model).mean().item()
                    
                    # Scale the value by the global warming value, if not 0
                    scaled_val = val / gw_val if gw_val != 0 else np.nan

                except Exception as e:
                    print(f"Could not extract {short_name} for model {model} in {f}: {e}")
                    val = np.nan
                    scaled_val = np.nan

                # Store the raw and scaled values
                regressor_values[short_name][model] = val
                regressor_values_scaled[short_name][model] = scaled_val

    # Find intersection of model sets for all variables
    model_sets = [set(driver.keys()) for driver in regressor_values.values()]
    common_models = sorted(set.intersection(*model_sets))
    
    # Build a dataframe from the collected values
    # models = sorted(model_set)
    data_raw = {sn: [regressor_values[sn].get(m, np.nan) for m in common_models] for sn in short_names}
    data_scaled = {sn: [regressor_values_scaled[sn].get(m, np.nan) for m in common_models] for sn in short_names}

    df_raw = pd.DataFrame(data_raw, index=common_models)
    df_scaled = pd.DataFrame(data_scaled, index=common_models)

    # Standardize the scaled data (z-score normalization)
    df_standardized = df_scaled.apply(stand, axis=0)

    # Create the output directory and save the dataframes as CSV
    out_dir = os.path.join(work_dir, "remote_drivers")
    os.makedirs(out_dir, exist_ok=True)

    df_raw.to_csv(os.path.join(out_dir, "drivers.csv"))
    df_scaled.to_csv(os.path.join(out_dir, "scaled_drivers.csv"))
    df_standardized.to_csv(os.path.join(out_dir, "scaled_standardized_drivers.csv"))

    print(f"Saved driver regressors to: {out_dir}")
    return df_raw, df_scaled, df_standardized

driver_config = dict(
        var_name=['psl', 'tas', 'psl', 'psl'],            # <- actual variable names in NetCDF
        short_name=['ubi', 'utas', 'esi', 'ctp'],           # <- names for regression/CSV outputs
        period1=['1960', '1979'],
        period2=['2070', '2099'],
        # season=[12, 1, 2],
        #box={'lat_min': -15, 'lat_max': 15, 'lon_min': -180, 'lon_max': 180}, # ta
        box={'lat_min': -90, 'lat_max': 90, 'lon_min': -180, 'lon_max': 180}, # pw
        work_dir='/climca/people/storylinetool/test_user/driver_test_outputs'
    )


df_raw, df_scaled, df_standardized = compute_drivers_from_netcdf(driver_config)